# Model Training

In [12]:
import cv2
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
import time
# NOTE: the next import is only valid 
# for scikit-learn version <= 0.17
# if you are using scikit-learn >= 0.18 then use this:
# from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split

%matplotlib inline

In [13]:
# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

In [14]:
# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [15]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

In [49]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, cspace='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCR_CB)
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features

In [50]:
# Read in car and non-car images
vehicle_images = glob.glob('./train_data/vehicles/**/*.png')
nonvehicle_images = glob.glob('./train_data/non-vehicles/**/*.png')

print('# of car images', len(vehicle_images))
print('# of non-car images', len(nonvehicle_images))

print(vehicle_images[0])

('# of car images', 8792)
('# of non-car images', 8968)
./train_data/vehicles/GTI_Far/image0000.png


In [51]:
test_image_dir = ['./test_images/test1.jpg']
feat = extract_features(test_image_dir, cspace='YCrCb', spatial_size=(16, 16),
                        hist_bins=32, orient=8, pix_per_cell=8, cell_per_block=2,
                        hog_channel='ALL')
test_image = mpimg.imread(test_image_dir[0])
print('dealing with a image size:', test_image.shape)
print('total number of features:', len(feat[0]))

('dealing with a image size:', (720, 1280, 3))
('total number of features:', 1359360)


In [53]:
cspace = 'YCrCb'
spatial_feat=True
hist_feat=True
hog_feat=True
spatial = (16,16)
hist = 32
orient = 8
pix_per_cell = 8
cell_per_block = 2
hog_channel = 'ALL'

car_features = extract_features(vehicle_images, cspace=cspace, spatial_size=spatial,
                                hist_bins=hist, orient=orient, pix_per_cell=pix_per_cell,
                                cell_per_block=cell_per_block, hog_channel=hog_channel)
notcar_features = extract_features(nonvehicle_images, cspace=cspace, spatial_size=spatial,
                                   hist_bins=hist, orient=orient, pix_per_cell=pix_per_cell,
                                   cell_per_block=cell_per_block, hog_channel=hog_channel)
print("Extraction Complete")

Extraction Complete


In [67]:
print("image sizes are", test_image_2.shape)
print("total number of rows (cars)", len(car_features))
print("total number of cols (cars)", len(car_features[0]))
test_image_2 = mpimg.imread(vehicle_images[0])

print("total number of rows (not cars)", len(notcar_features))
print("total number of cols (not cars)", len(notcar_features[0]))

('image sizes are', (64, 64, 3))
('total number of rows (cars)', 8792)
('total number of cols (cars)', 5568)
('total number of rows (not cars)', 8968)
('total number of cols (not cars)', 5568)


In [68]:
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

In [69]:
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

(16.87, 'Seconds to train SVC...')
('Test Accuracy of SVC = ', 0.9873)


In [70]:
print('X_train.shape',X_train.shape)
print('X_test.shape',X_test.shape)

('X_train.shape', (14208, 5568))
('X_test.shape', (3552, 5568))


In [77]:
import os
import pickle

In [75]:
parameters = {'cspace': cspace,
              'orient': orient,
              'pix_per_cell': pix_per_cell,
              'cell_per_block': cell_per_block,
              'hog_channel': hog_channel,
              'spatial_size': spatial,
              'hist_bins': hist,
              'spatial_feat': spatial_feat,
              'hist_feat': hist_feat,
              'hog_feat': hog_feat}


In [80]:
pickle_file_features = 'features.pickle'
if not os.path.isfile(pickle_file_features):
    print('Saving data to pickle file')
    try:
        with open(pickle_file_features, 'wb') as pfile:
            pickle.dump(
                {
                    'train_dataset': X_train,
                    'train_labels': y_train,
                    'test_dataset': X_test,
                    'test_labels': y_test,
                    'X_scaler': X_scaler,
                    'parameters': parameters
                },
                pfile, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save data to', pickle_file_features, ':', e)
        raise

Saving data to pickle file...


In [82]:
pickle_file_svc = 'svc.pickle'
if not os.path.isfile(pickle_file_svc):
    print('Saving data to pickle file')
    try:
        with open(pickle_file_svc, 'wb') as pfile:
            pickle.dump(
                {
                    'svc': svc,
                    'X_scaler': X_scaler,
                    'parameters': parameters
                },
                pfile, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save data to', pickle_file_svc, ':', e)
        raise

Saving data to pickle file


In [83]:
loaded_svc = pickle.load( open( "svc.pickle", "rb" ) )

In [84]:
print(loaded_svc)

{'X_scaler': StandardScaler(copy=True, with_mean=True, with_std=True), 'svc': LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0), 'parameters': {'spatial_size': (16, 16), 'pix_per_cell': 8, 'cell_per_block': 2, 'hog_feat': True, 'hog_channel': 'ALL', 'cspace': 'YCrCb', 'hist_feat': True, 'hist_bins': 32, 'orient': 8, 'spatial_feat': True}}


In [93]:
print('Test Accuracy of original SVC = ', round(svc.score(X_test, y_test), 4))
print('Test Accuracy of pickled SVC = ', round(loaded_svc['svc'].score(X_test, y_test), 4))
# pickle files have been saved correctly! moving on..

('Test Accuracy of original SVC = ', 0.9873)
('Test Accuracy of pickled SVC = ', 0.9873)
